<a href="https://colab.research.google.com/github/AI-Lab-2025-2-3rd/ai-project-hyeongnim/blob/Do0o0ony/%EC%98%88%EC%95%84~.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/KUBIG/DL/KUBIG_25-W STUDY/제출용/data"

# data loading

- 원본 데이터 : <br>
TRAIN = pd.read_csv(path+"/train.csv")<br>
TEST = pd.read_csv(path+"/test.csv")<br>

- 원본 데이터 전처리 : <br>
TRAIN_prep = pd.read_csv(path+"/TRAIN_prep.csv")<br>
TEST_prep = pd.read_csv(path+"/TEST_prep.csv")<br>


- train/valid split : <br>
train_model = pd.read_csv(path+"/train_model.csv") <br>
valid_model = pd.read_csv(path+"/valid_model.csv") <br>

- train/valid split : <br>
train_prep = pd.read_csv(path+"/train_prep.csv") <br>
valid_prep = pd.read_csv(path+"/valid_prep.csv") <br>

In [ ]:
import pandas as pd
TRAIN = pd.read_csv(path+"/train.csv")
TEST = pd.read_csv(path+"/test.csv")

# Preprocessing + Encoding

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm

In [ ]:
def preprocessing(text):

    start_code = 0xAC00 # 한글 시작 유니코드
    end_code = 0xD7A3 # 한글 끝 유니코드

    # 초성, 중성, 종성 리스트
    cho = [chr(i) for i in range(0x1100, 0x1113)] # 초성 유니코드
    jung = [chr(i) for i in range(0x1161, 0x1176)] # 중성 유니코드
    jong = [chr(i) for i in range(0x11A8, 0x11C3)] # 종성 유니코드

    chos = []
    jungs = []
    jongs = []
    positions = []

    cleaned_text = re.sub(r'[^\uAC00-\uD7A3\s]', '', text)  # 한글과 공백만 남기기
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # 중복 공백 제거


    for char in cleaned_text:
        if start_code <= ord(char) <= end_code:
            char_code = ord(char) - start_code
            cho_idx = char_code // (21 * 28)
            jung_idx = (char_code % (21 * 28)) // 28
            jong_idx = char_code % 28

            # 초성, 중성
            chos.append(cho[cho_idx])
            jungs.append(jung[jung_idx])

            # 종성 (없을 경우 'ㅉ'토큰; 받침에 올 수 없는 쌍자음)
            jongs.append(jong[jong_idx - 1] if jong_idx > 0 else 'ㅉ')
        else:
            # 한글이 아닌 경우는 무시
            continue

    word_list = cleaned_text.split()  # 띄어쓰기 기준으로 단어 분리
    for word in word_list:
        for idx, _ in enumerate(word):
            relative_pos = (idx + 1) / len(word)  # 현재 음절의 상대적 위치 계산
            positions.append(round(relative_pos,2))

    return chos, jungs, jongs, positions

def extract_kr_char(text):
    kr_chars = re.findall(r'[가-힣]', text)
    return kr_chars


def Encoder(data, jamo_to_index):
  encoded_data = []
  for jamos in data:
    index_sequences = []
    for jamo in jamos:
      index_sequences.append(jamo_to_index[jamo])
    encoded_data.append(index_sequences)
  return encoded_data

def preprocessing_encoding(df, train_flg = True):
  df_prep = pd.DataFrame()
  data = df.copy()

  # 초성, 중성, 종성 유니코드 범위
  cho = [chr(i) for i in range(0x1100, 0x1113)]  # 초성 (ㄱ~ㅎ)
  jung = [chr(i) for i in range(0x1161, 0x1176)]  # 중성 (ㅏ~ㅣ)
  jong = [chr(i) for i in range(0x11A8, 0x11C3)]  # 종성 (ㄱ~ㅄ)
  jamo = cho + jung + jong + ['ㅉ']

  jamo_to_index = {jamo[i]:i for i in range(len(jamo))}
  index_to_jamo = {i:jamo[i] for i in range(len(jamo))}

  for i in tqdm(range(len(data))):
    chos, jungs, jongs, poss = preprocessing(data['input'][i])

    chars = extract_kr_char(data['input'][i])

    if train_flg:
      orgs = extract_kr_char(data['output'][i])
      chos_, jungs_, jongs_, _ = preprocessing(data['output'][i])

      temp = pd.DataFrame({'index':i, '원본':orgs,'난독화':chars, '위치': poss,
                        '초성':chos_, '중성': jungs_, '종성':jongs_,
                        '난독화_초성':chos, '난독화_중성': jungs, '난독화_종성':jongs})
    else:
      temp = pd.DataFrame({'index':i,'난독화':chars, '위치': poss,
                        '난독화_초성':chos, '난독화_중성': jungs, '난독화_종성':jongs})
    df_prep = pd.concat([df_prep,temp])
    df_prep.reset_index(drop=True, inplace=True)

  if train_flg:
    n = 4
  else:
    n = 3

  data = df_prep.iloc[:,n:].values
  encoded_data = Encoder(data, jamo_to_index)
  encoded_data = np.array(encoded_data)

  df_encoded = pd.DataFrame(encoded_data,
                             columns=[f'{col}_ENC'for col in df_prep.columns[n:]])
  df_prep = pd.concat([df_prep, df_encoded], axis=1)

  return df_prep

In [ ]:
TRAIN_prep = preprocessing_encoding(TRAIN)
TEST_prep = preprocessing_encoding(TEST, train_flg = False)

In [ ]:
TRAIN_prep.to_csv(path+'/TRAIN_prep.csv', index=False)
TEST_prep.to_csv(path+'/TEST_prep.csv', index=False)

## 모델 훈련을 위한 train, valid split

In [ ]:
TRAIN = pd.read_csv(path+"/train.csv")
TEST = pd.read_csv(path+"/test.csv")

In [ ]:
X_train, y_train = TRAIN.iloc[:,:-1], TRAIN.iloc[:,-1]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
train_model = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
valid_model = pd.concat([X_valid, y_valid], axis=1).reset_index(drop=True)

In [ ]:
train_model.to_csv(path+'/train_model.csv', index=False)
valid_model.to_csv(path+'/valid_model.csv', index=False)

- 전처리

In [ ]:
train_prep = preprocessing_encoding(train_model)
valid_prep = preprocessing_encoding(valid_model)

In [ ]:
train_prep.to_csv(path+'/train_prep.csv', index=False)
valid_prep.to_csv(path+'/valid_prep.csv', index=False)

# 1차 해독 (MLP)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import random
import warnings
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import MultiStepLR
import torch.optim as optim

warnings.filterwarnings('ignore')
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

## MLP training
- AdamW + LR Scheduler

In [ ]:
train_prep = pd.read_csv(path+'/train_prep.csv')
valid_prep = pd.read_csv(path+'/valid_prep.csv')

In [ ]:
X_train, y_train = train_prep.iloc[:,13:].values, train_prep.iloc[:,10:13].values
X_valid, y_valid = valid_prep.iloc[:,13:].values, valid_prep.iloc[:,10:13].values

X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
X_valid = torch.tensor(X_valid, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.long)

In [ ]:
X1_train = X_train
X2_train = torch.tensor(train_prep.iloc[:,3].values, dtype = torch.float)
X1_valid = X_valid
X2_valid = torch.tensor(valid_prep.iloc[:,3].values, dtype = torch.float)

In [ ]:
class Embedding(nn.Module):
    def __init__(self,embedding_dim, num_jamo=68):
        super(Embedding, self).__init__()
        self.jamo_embedding = nn.Embedding(num_jamo, embedding_dim//3)
        self.pos_embedding = nn.Sequential(
            nn.Linear(1, 8),
            nn.ReLU(),
            nn.Linear(8, embedding_dim%3)
            )

    def forward(self, x1, x2):
        x1 = self.jamo_embedding(x1)
        x1 = x1.view(x1.size(0), -1)

        x2 = x2.unsqueeze(-1)
        x2 = self.pos_embedding(x2)

        x = torch.cat([x2, x1], dim=-1)
        return x

class MLP(nn.Module):
  def __init__(self, embedding_dim, hidden_dim):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(embedding_dim, hidden_dim)
    self.bn1 = nn.BatchNorm1d(hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, hidden_dim)
    self.bn2 = nn.BatchNorm1d(hidden_dim)

  def forward(self, x):
    x = self.fc1(x)
    x = self.bn1(x)
    x = torch.relu(x)
    x = self.fc2(x) + x # skip connection
    x = self.bn2(x)
    x = torch.tanh(x)
    return x
class Decoder(nn.Module):
  def __init__(self, hidden_dim = 128, embedding_dim = 64):
    super(Decoder, self).__init__()
    self.embedding = Embedding(embedding_dim)
    self.mlp = MLP(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, 68)

  def forward(self, x1, x2):
    x = self.embedding(x1,x2)
    x = self.mlp(x)
    x = self.fc(x)
    return x

def set_seed(seed: int = 42):
    random.seed(seed)  # Python의 random 모듈 시드 고정
    np.random.seed(seed)  # NumPy 시드 고정
    torch.manual_seed(seed)  # PyTorch의 시드 고정
    torch.cuda.manual_seed(seed)  # CUDA 시드 고정 (GPU 사용 시)
    torch.cuda.manual_seed_all(seed)  # 다중 GPU 환경 시드 고정
    torch.backends.cudnn.deterministic = True  # CUDNN 연산을 결정적으로 설정
    torch.backends.cudnn.benchmark = False  # 연산 최적화 비활성화 (재현성 보장)

set_seed(42)  # 원하는 시드 값 설정

class CustomDataset(TensorDataset):
    def __init__(self, jamo, position, labels):

        self.jamo = jamo
        self.position = position
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        jamo_sample = self.jamo[idx]
        pos_sample = self.position[idx]
        label = self.labels[idx]
        return jamo_sample, pos_sample, label


### 훈련 / 검증 데이터 분할 학습
인사이트 도출용이므로 주석처리하고 넘어가도 됨

In [ ]:
# from torch.optim import AdamW
# from torch.utils.data import DataLoader, TensorDataset

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = Decoder(hidden_dim = 256, embedding_dim = 128).to(device)
# criterion = nn.CrossEntropyLoss()

# train_dataset = CustomDataset(X1_train, X2_train, y_train)
# valid_dataset = CustomDataset(X1_valid, X2_valid, y_valid)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [ ]:
# import torch.optim as optim

# optimizer = AdamW(model.parameters(), lr=0.001, weight_decay = 0.01)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
# # 10 epoch 동안 val_loss가 감소하지 않으면, 학습률 0.5 감소
# model.train()

# epochs = 200
# loss_curve = np.zeros((2,epochs))
# best_val_loss = float('inf')
# save_path = path + "/best_model.pth"

# for epoch in range(epochs):
#     total_loss = 0
#     # 학습
#     for batch_idx, (jamo, pos, target) in enumerate(train_loader):
#         optimizer.zero_grad()
#         jamo, pos, target = jamo.to(device), pos.to(device), target.to(device)
#         logits = model(jamo, pos)

#         logit_cho = logits[:, :19]
#         logit_jung = logits[:, 19:40]
#         logit_jong = logits[:, 40:]

#         loss1 = criterion(logit_cho, target[:, 0])
#         loss2 = criterion(logit_jung, target[:, 1] - 19)
#         loss3 = criterion(logit_jong, target[:, 2] - 40)
#         loss = loss1 + loss2 + loss3

#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#         if batch_idx % 4500 == 0:
#             print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}')

#     print(f'====> Epoch: {epoch}, Total Loss: {total_loss / (len(X_train) // 64):.4f}')
#     loss_curve[0,epoch] = total_loss / (len(X_train) // 64)

#     # 검증
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for jamo, pos, target in valid_loader:
#             jamo, pos, target = jamo.to(device), pos.to(device), target.to(device)
#             logits = model(jamo, pos)

#             logit_cho = logits[:, :19]
#             logit_jung = logits[:, 19:40]
#             logit_jong = logits[:, 40:]

#             loss1 = criterion(logit_cho, target[:, 0])
#             loss2 = criterion(logit_jung, target[:, 1] - 19)
#             loss3 = criterion(logit_jong, target[:, 2] - 40)
#             loss = loss1 + loss2 + loss3

#             val_loss += loss.item()

#     val_loss /= len(valid_loader)
#     print(f'====> Validation Loss: {val_loss:.4f}')
#     loss_curve[1,epoch] = val_loss
#     scheduler.step(val_loss) # 스케줄러 업데이트

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'val_loss': best_val_loss,
#         }, save_path)
#         print(f"Model Saved. (epoch: {epoch}, val_loss: {val_loss:.4f})")
#     model.train()


In [ ]:
# plt.plot(loss_curve[0], label='train')
# plt.plot(loss_curve[1], label='valid')
# plt.legend()
# plt.show()

### 훈련 데이터 전체 학습

In [ ]:
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Decoder(hidden_dim = 256, embedding_dim = 128).to(device)
criterion = nn.CrossEntropyLoss()

X1_train = torch.vstack([X1_train, X1_valid])
X2_train = torch.hstack([X2_train, X2_valid])
y_train = torch.vstack([y_train, y_valid])

train_dataset = CustomDataset(X1_train, X2_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
optimizer = AdamW(model.parameters(), lr=0.001, weight_decay = 0.01)
scheduler = MultiStepLR(optimizer, milestones = range(45,165,15), gamma=0.5)
model.train()

epochs = 200
loss_curve = np.zeros((1,epochs))

for epoch in range(epochs):
    total_loss = 0
    # 학습
    for batch_idx, (jamo, pos, target) in enumerate(train_loader):
        optimizer.zero_grad()
        jamo, pos, target = jamo.to(device), pos.to(device), target.to(device)
        logits = model(jamo, pos)

        logit_cho = logits[:, :19]
        logit_jung = logits[:, 19:40]
        logit_jong = logits[:, 40:]

        loss1 = criterion(logit_cho, target[:, 0])
        loss2 = criterion(logit_jung, target[:, 1] - 19)
        loss3 = criterion(logit_jong, target[:, 2] - 40)
        loss = loss1 + loss2 + loss3

        lr = optimizer.param_groups[0]['lr']

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 4500 == 0:
          lr = optimizer.param_groups[0]['lr']
          print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}, LR: {lr}')

    print(f'====> Epoch: {epoch}, Total Loss: {total_loss / (len(X1_train) // 64):.4f}')
    loss_curve[0,epoch] = total_loss / (len(X_train) // 64)
    scheduler.step()

    if epoch + 1 in range(50,201,10):
      save_path = f"{path}/full_model_{epoch+1}.pth"
      torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, save_path)
      print(f"Model Saved. (epoch: {epoch+1})")

In [ ]:
plt.plot(loss_curve[0], label='train')
plt.legend()
plt.show()

## 해독

In [ ]:
TRAIN = pd.read_csv(path+'/train.csv') #원본
TRAIN_prep = pd.read_csv(path+'/TRAIN_prep.csv') #원본
TEST = pd.read_csv(path+'/test.csv')
TEST_prep = pd.read_csv(path+'/TEST_prep.csv')

train = pd.read_csv(path+'/train_model.csv')
train_prep = pd.read_csv(path+'/train_prep.csv')
valid = pd.read_csv(path+'/valid_model.csv')
valid_prep = pd.read_csv(path+'/valid_prep.csv')

In [ ]:
cho = [chr(i) for i in range(0x1100, 0x1113)]  # 초성 (ㄱ~ㅎ)
jung = [chr(i) for i in range(0x1161, 0x1176)]  # 중성 (ㅏ~ㅣ)
jong = [chr(i) for i in range(0x11A8, 0x11C3)]  # 종성 (ㄱ~ㅄ)
jamo = cho + jung + jong + ['ㅉ']

jamo_to_index = {jamo[i]:i for i in range(len(jamo))}
index_to_jamo = {i:jamo[i] for i in range(len(jamo))}

In [ ]:
def combine_jamos(choseong_str, jungseong_str, jongseong_str='ㅉ'):

    # 초성, 중성 인덱스 계산
    choseong_index = cho.index(choseong_str)
    jungseong_index = jung.index(jungseong_str)

    # 종성 인덱스 계산 (받침 없는 경우 0으로 처리)
    if jongseong_str == 'ㅉ':  # 받침이 없는 경우
        jongseong_index = 0
    else:
        jongseong_index = jong.index(jongseong_str) + 1  # 종성은 1부터 시작

    # 유니코드 한글 음절 계산
    HANGUL_BASE = 0xAC00
    syllable_code = (
        HANGUL_BASE
        + choseong_index * 21 * 28  # 초성
        + jungseong_index * 28      # 중성
        + jongseong_index           # 종성
    )

    return chr(syllable_code)


In [ ]:
def inference(model, data, data_prep, index_to_jamo = index_to_jamo):

  X1 = data_prep[['난독화_초성_ENC','난독화_중성_ENC','난독화_종성_ENC']].values
  X2 = data_prep['위치'].values
  X1, X2 = torch.tensor(X1, dtype = torch.long), torch.tensor(X2, dtype = torch.float)

  model.eval()
  model.to('cpu')

  with torch.no_grad():
    logits = model(X1, X2)
    logit_cho = logits[:,:19]
    logit_jung = logits[:,19:40]
    logit_jong = logits[:,40:]

  pred_cho = torch.argmax(logit_cho, dim=1)
  pred_jung = torch.argmax(logit_jung, dim=1) + 19
  pred_jong = torch.argmax(logit_jong, dim=1) + 40
  pred_jamo = torch.stack([pred_cho, pred_jung, pred_jong], dim=1)

  pred = pd.DataFrame(pred_jamo, columns=['초성', '중성', '종성'])
  pred['결과'] = pred.apply(lambda x : combine_jamos(index_to_jamo[x['초성']],
                                                   index_to_jamo[x['중성']],
                                                   index_to_jamo[x['종성']]), axis =1)


  chos = [chr(i) for i in range(0x1100, 0x1113)] # 초성 유니코드
  jungs = [chr(i) for i in range(0x1161, 0x1176)] # 중성 유니코드
  jongs = [chr(i) for i in range(0x11A8, 0x11C3)] # 종성 유니코드
  jamo = chos + jungs + jongs + ['ㅉ']

  jamo_to_index = {jamo[i]:i for i in range(len(jamo))}
  index_to_jamo = {i:jamo[i] for i in range(len(jamo))}

  decoded_sentences = []
  i = 0

  for _, sentence in data.iterrows():
    decoded_sentence = ''
    for char in sentence['input']:
      if 0xAC00 <= ord(char) <= 0xD7A3:
        decoded_sentence += pred.loc[i, '결과']
        i += 1

      else:
        decoded_sentence += char
    decoded_sentences.append(decoded_sentence)

  return decoded_sentences, pred_jamo

In [ ]:
save_path = path+"/full_model_140.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load(save_path)
model_ = Decoder(embedding_dim = 128, hidden_dim = 256).to(device)
model_.load_state_dict(checkpoint['model_state_dict'])
model_.to('cpu')

def compute_accuracy(data, pred):
  acc1 = (data.iloc[:,10:13].values == pred).all(axis=1).sum()/len(data)
  acc2 = (data.iloc[:,10:13].values == pred).reshape(-1,).sum()/len(data)/3
  return acc1, acc2

In [ ]:
import pandas as pd
import numpy as np

def process_and_save_data(model, data, data_prep, dataset_type, path = path):
    result, pred = inference(model, data, data_prep)
    if dataset_type in ['train','TRAIN', 'valid']:
      dec1_df = pd.DataFrame({
        'input': data['input'],
        'input_DEC1': result,
        'output': data['output']
        })
    else:
      dec1_df = pd.DataFrame({
        'input': data['input'],
        'input_DEC1': result
        })

    dec1_df.to_csv(path + f"/{dataset_type}_dec1.csv", index=False)
    return dec1_df

process_and_save_data(model_, TRAIN, TRAIN_prep, 'TRAIN')
process_and_save_data(model_, train, train_prep, 'train')
process_and_save_data(model_, valid, valid_prep, 'valid')
process_and_save_data(model_, TEST, TEST_prep, 'TEST')

# 2차 해독 _electra

In [ ]:
import pandas as pd
train = pd.read_csv(path+"/TRAIN_dec1.csv")
test = pd.read_csv(path+"/TEST_dec1.csv")

## Chunking

In [ ]:
import re

def clean_text(text):
    # 1. 한국어(가-힣), 영어(a-zA-Z), 숫자(0-9), 공백(\s) 외 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s(),.!?\'\"\[\]\{\}:]', '', text)

    # 2. ㅋㅋ, ㅜㅜ, ㅠㅠ, ㅎㅎ 등과 같은 불완전한 한글 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)

    return text

def split_sentences(text):
    # 온점, 느낌표, 물음표 뒤에 공백이 있으면 모두 분리
    sentences = re.split(r'(?<=[.!?])\s', text)
    return [sentence for sentence in sentences if sentence]

def clean_blank(text):
    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환
    return text

In [ ]:
train['cleaned_input'] = train['input_DEC1'].apply(clean_text)
test['cleaned_input'] = test['input_DEC1'].apply(clean_text)
train['cleaned_output'] = train['output'].apply(clean_text)

In [ ]:
train['chunked_input'] = train['cleaned_input'].apply(split_sentences)
test['chunked_input'] = test['cleaned_input'].apply(split_sentences)
train['chunked_output'] = train['cleaned_output'].apply(split_sentences)

train_exploded = train.explode('chunked_input')[['chunked_input']].reset_index(drop=False)
train_exploded_ = train.explode('chunked_output')[['chunked_output']].reset_index(drop=True)

test_chunked = test.explode('chunked_input')[['chunked_input']].reset_index(drop=False)
train_chunked = pd.concat([train_exploded, train_exploded_], axis=1)

In [ ]:
train_chunked['chunked_input'].apply(len).sort_values(ascending=False)

In [ ]:
train_chunked.loc[2435]

In [ ]:
train_chunked[train_chunked['index'] == 658]

In [ ]:
# 청킹 후에도 긴 문장에 대해 수동 분할
input1, input2 = train_chunked.iloc[2435][1][:423], train_chunked.iloc[2435][1][424:]
input3 = train_chunked.iloc[2436][1]
output1, output2 = train_chunked.iloc[2435][2][:423], train_chunked.iloc[2435][2][424:]
output3 = train_chunked.iloc[2436][2]
temp = pd.DataFrame({'index':[658,658,658], 'chunked_input':[input1,input2,input3],
                     'chunked_output':[output1,output2,output3]})
train_chunked.drop(index=[2435,2436], inplace=True)
train_chunked = pd.concat([train_chunked, temp]).reset_index(drop=True)

In [ ]:
train_chunked

In [ ]:
train_chunked['chunked_input'] = train_chunked.apply(lambda x: clean_blank(x['chunked_input']), axis = 1)
train_chunked['chunked_output'] = train_chunked.apply(lambda x: clean_blank(x['chunked_output']), axis = 1)
train_chunked['len'] = train_chunked['chunked_input'].apply(len)
train_chunked.drop(train_chunked[train_chunked['len'] == 0].index, inplace=True)
train_chunked.reset_index(drop=True, inplace=True)

test_chunked['chunked_input'] = test_chunked.apply(lambda x: clean_blank(x['chunked_input']), axis = 1)
test_chunked['len'] = test_chunked['chunked_input'].apply(len)
test_chunked.drop(test_chunked[test_chunked['len'] == 0].index, inplace=True)
test_chunked.reset_index(drop=True, inplace=True)

In [ ]:
train_chunked.drop(columns=['len'], inplace=True)
test_chunked.drop(columns=['len'], inplace=True)

In [ ]:
train_chunked.to_csv(path+"/TRAIN_dec1_chunked.csv", index=False)
test_chunked.to_csv(path+"/TEST_dec1_chunked.csv", index=False)

In [ ]:
train_chunked = pd.read_csv(path+"/TRAIN_dec1_chunked.csv")
test_chunked = pd.read_csv(path+"/TEST_dec1_chunked.csv")

## Utils

In [ ]:
import csv
import pandas as pd
import json

In [ ]:
def restore_original_text(original_text, decoded_text):

    restored_text = ""
    decoded_idx = 0
    # 한글, 영어, 숫자, 공백, 문장부호만 허용하는 정규식
    valid_char_pattern = re.compile(r'[가-힣a-zA-Z0-9(),.!?\'\"\[\]\{\}:]')


    for char in original_text:
        if valid_char_pattern.match(char):
            # 유효한 문자일 경우, 해독된 문자로 대체
            if decoded_idx < len(decoded_text):
                restored_text += decoded_text[decoded_idx]
                decoded_idx += 1
            else:
                # 만약 해독된 텍스트가 끝났다면 원문 문자 그대로 추가
                restored_text += char
        else:
            # 특수문자 및 불완전한 한글은 원문 그대로 유지
            restored_text += char

    return restored_text

In [ ]:
# CSV 파일을 JSON 파일로 변환하는 함수
def csv_to_json(csv_file_path, json_file_path):
    with open(csv_file_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        data = list(reader)

    # JSON 파일 저장
    with open(json_file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

csv_file_path = path+"/TRAIN_dec1_chunked.csv"
json_file_path = path+"/TRAIN_dataset.json"  # 변환할 JSON 파일 경로

csv_to_json(csv_file_path, json_file_path)
print(f"변환 완료: {json_file_path}")

In [ ]:
import sentencepiece as spm
import os
# SentencePiece 모델 학습 (음절 단위)
os.chdir(path)
spm.SentencePieceTrainer.Train(f'--input=TRAIN_dataset.json --model_prefix=syllable --vocab_size=12000 --character_coverage=1.0 --model_type=char')

# SentencePiece 모델 로드
sp = spm.SentencePieceProcessor()
sp.Load("syllable.model")

## Training

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv(path+"/TRAIN_dec1.csv")
test = pd.read_csv(path+"/TEST_dec1.csv")

train_chunked = pd.read_csv(path+"/TRAIN_dec1_chunked.csv")
test_chunked = pd.read_csv(path+"/TEST_dec1_chunked.csv")

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
import sentencepiece as spm

# 기존 KoELECTRA 모델의 토크나이저 로드
base_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

tokenizer_path = path+"/syllable.model"
vocab_path = path+"/syllable.vocab"

# SentencePiece를 이용한 커스텀 토크나이저 적용
class CustomTokenizer(ElectraTokenizer):
    def __init__(self, sp_model_path, *args, **kwargs):
        super().__init__(vocab_file=vocab_path, *args, **kwargs)
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(sp_model_path)

    def tokenize(self, text):
        return self.sp_model.EncodeAsPieces(text)

    def convert_tokens_to_ids(self, tokens):
        return [self.sp_model.PieceToId(token) for token in tokens]

    def convert_ids_to_tokens(self, ids):
        return [self.sp_model.IdToPiece(id) for id in ids]

# Custom Tokenizer 생성
custom_tokenizer = CustomTokenizer(tokenizer_path)

# KoELECTRA 모델 로드
model = ElectraForMaskedLM.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [ ]:
def restore_original_text(original_text, decoded_text):

    restored_text = ""
    decoded_idx = 0  # 해독된 텍스트에서 현재 참조 중인 문자 위치

    # 한글, 영어, 숫자, 공백, 문장부호만 허용하는 정규식
    valid_char_pattern = re.compile(r'[가-힣a-zA-Z0-9(),.!?\'\"\[\]\{\}:]')


    for char in original_text:
        if valid_char_pattern.match(char):
            # 유효한 문자일 경우, 해독된 문자로 대체
            if decoded_idx < len(decoded_text):
                restored_text += decoded_text[decoded_idx]
                decoded_idx += 1
            else:
                # 만약 해독된 텍스트가 끝났다면 원문 문자 그대로 추가
                restored_text += char
        else:
            # 특수문자 및 불완전한 한글은 원문 그대로 유지
            restored_text += char

    return restored_text

def clean_text(text):
    # 1. 한국어(가-힣), 영어(a-zA-Z), 숫자(0-9), 공백(\s) 외 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s(),.!?\'\"\[\]\{\}:]', '', text)

    # 2. ㅋㅋ, ㅜㅜ, ㅠㅠ, ㅎㅎ 등과 같은 불완전한 한글 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)

    return text

def clean_blank(text):
    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환
    return text

- Dataloader

In [ ]:
import torch
import csv
import json
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스 정의
class ObfuscatedDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=512):
        with open(data_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_length = max_length  # 최대 길이 지정

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_tokens = self.tokenizer.tokenize(item["chunked_input"])
        target_tokens = self.tokenizer.tokenize(item["chunked_output"])

        # 토큰을 ID로 변환
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)
        target_ids = self.tokenizer.convert_tokens_to_ids(target_tokens)

        # **고정된 max_length로 패딩**
        input_ids = input_ids[:self.max_length]
        target_ids = target_ids[:self.max_length]

        input_ids += [0] * (self.max_length - len(input_ids))  # 패딩 추가
        target_ids += [0] * (self.max_length - len(target_ids))

        # **어텐션 마스크 생성 (패딩된 부분: 0, 나머지: 1)**
        attention_mask = [1 if token != 0 else 0 for token in target_ids]

        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long), torch.tensor(attention_mask, dtype=torch.long)

# 데이터셋 로드
train_dataset = ObfuscatedDataset(path+"/TRAIN_dataset.json", custom_tokenizer, max_length=512)

# DataLoader 설정
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


- loss & Optimizer

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
from transformers import get_linear_schedule_with_warmup

num_epochs = 20
total_steps = num_epochs * len(train_dataloader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

loss_curve = np.zeros((1,num_epochs))

- Training

In [ ]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python의 random 모듈 시드 고정
    np.random.seed(seed)  # NumPy 시드 고정
    torch.manual_seed(seed)  # PyTorch의 시드 고정
    torch.cuda.manual_seed(seed)  # CUDA 시드 고정 (GPU 사용 시)
    torch.cuda.manual_seed_all(seed)  # 다중 GPU 환경 시드 고정
    torch.backends.cudnn.deterministic = True  # CUDNN 연산을 결정적으로 설정
    torch.backends.cudnn.benchmark = False  # 연산 최적화 비활성화 (재현성 보장)

set_seed(42)  # 원하는 시드 값 설정

In [ ]:
for epoch in range(num_epochs):

    model.train()
    total_train_loss = 0
    save_path = path+f"/full_electra_epc{epoch+1}.pth"

    for batch_idx, (input_ids, target_ids, attention_mask) in enumerate(train_dataloader):
        input_ids, target_ids, attention_mask = (
            input_ids.to(device),
            target_ids.to(device),
            attention_mask.to(device),
        )

        outputs = model(input_ids, attention_mask=attention_mask).logits
        loss = criterion(outputs.view(-1, outputs.size(-1)), target_ids.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_train_loss += loss.item()

        # 100 배치마다 진행 상황 출력
        if (batch_idx + 1) % 100 == 0:
            avg_loss = total_train_loss / (batch_idx + 1)
            print(f"[Train] Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_dataloader)}], Loss: {avg_loss:.4f}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    loss_curve[0,epoch] = avg_train_loss
    print(f"[Train] Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_train_loss:.4f}")

    if epoch >= 5:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, save_path)
        print(f"Model Saved. (epoch: {epoch+1})")



In [ ]:
plt.plot(loss_curve[0][1:], label='train')
plt.legend()
plt.show()

## 2차 해독

In [ ]:
train = pd.read_csv(path+"/TRAIN_dec1.csv")
test = pd.read_csv(path+"/TEST_dec1.csv")
train_chunked = pd.read_csv(path+"/TRAIN_dec1_chunked.csv")
test_chunked = pd.read_csv(path+"/TEST_dec1_chunked.csv")

In [ ]:
import torch
from transformers import ElectraForMaskedLM
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ElectraForMaskedLM.from_pretrained("monologg/koelectra-base-v3-discriminator")
checkpoint = torch.load(path+"/full_electra_epc20.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

def denoise_text(input_text, tokenizer, max_length=512):

    input_tokens = tokenizer.tokenize(input_text)
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

    input_ids = input_ids[:max_length]
    attention_mask = [1] * len(input_ids)
    input_ids += [0] * (max_length - len(input_ids))  # 패딩
    attention_mask += [0] * (max_length - len(attention_mask))

    input_ids_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    attention_mask_tensor = torch.tensor([attention_mask], dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(input_ids_tensor, attention_mask=attention_mask_tensor).logits
        predicted_ids = torch.argmax(outputs, dim=-1)[0].cpu().tolist()

    final_ids = [
        pred if a == 0 else (pred if pred != 0 else 59)
        for orig, pred, a in zip(input_ids, predicted_ids, attention_mask)
    ]

    predicted_tokens = tokenizer.convert_ids_to_tokens(final_ids)
    restored_text = "".join(predicted_tokens).replace("▁", " ")

    return restored_text

def check_double_blanks(text):
    return bool(re.search(r' {2,}', text))

def check_repeated_last_char(text):
    return bool(re.search(r'([가-힣])\1+$', text))


def remove_trailing_unks(text):

    cleaned_text = re.sub(r'(<unk>\s*)+$', '', text)
    return cleaned_text

def post_processing(data, data_chunked, result, test_flg = True):
  temp = data.copy()
  temp_chunked = data_chunked.copy()

  for i in range(len(result)):
    result[i] = remove_trailing_unks(result[i])

  temp_chunked['input_DEC2'] = result
  data_recovered = temp_chunked.groupby("index").agg(list).reset_index(drop=True)
  if test_flg:
    data_recovered.columns = ['input_DEC1','input_DEC2']
    data_recovered['input_DEC2'] = data_recovered['input_DEC2'].apply(lambda x: ' '.join(x))
  else:
    data_recovered.columns = ['input_DEC1','output','input_DEC2']
    data_recovered['input_DEC2'] = data_recovered['input_DEC2'].apply(lambda x: ' '.join(x))

  temp['input_DEC1'] = temp.apply(lambda x: clean_blank(x['input_DEC1']), axis = 1)

  temp['input_DEC2'] = data_recovered['input_DEC2']
  temp['input_DEC2'] = temp.apply(lambda x: clean_blank(x['input_DEC2']), axis = 1)
  temp['input_DEC2'] = temp.apply(lambda x: x['input_DEC2'].replace(" ",""), axis= 1)

  temp['input_DEC2'] = temp.apply(lambda x: restore_original_text(x['input_DEC1'],x['input_DEC2']), axis = 1)

  return temp

In [ ]:
from tqdm import tqdm
test_sentences = test_chunked['chunked_input']

decoded = []
for sentence in tqdm(test_sentences):
    restored = denoise_text(sentence, custom_tokenizer)
    decoded.append(restored)

test_result = post_processing(test, test_chunked, decoded)

In [ ]:
test_result.to_csv(path+"/TEST_dec2.csv", index=False)

In [ ]:
submission = pd.read_csv(path+"/sample_submission.csv")
submission['output'] = test_result['input_DEC2']
submission.to_csv(path+"/submission_electra.csv", index=False)

# 2차 해독_후처리

In [ ]:
import pandas as pd
import re
# 잘 학습된 ELECTRA로 추론된 test dec2 셋
test = pd.read_csv(path+"/TEST_dec2.csv")

In [ ]:
def clean_text(text):
    # 1. 한국어(가-힣), 영어(a-zA-Z), 숫자(0-9), 공백(\s) 외 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s(),.!?\'\"\[\]\{\}:]', '', text)

    # 2. ㅋㅋ, ㅜㅜ, ㅠㅠ, ㅎㅎ 등과 같은 불완전한 한글 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)

    return text

def split_sentences(text):
    # 기본적인 문장 분리
    sentences = re.split(r'(?<=[.!?])\s', text)
    sentences = [sentence.strip() for sentence in sentences if sentence]  # 공백 제거 및 빈 요소 제거

    merged_sentences = []
    temp = ""
    open_bracket_count = 0  # 열린 괄호 개수

    for i, sentence in enumerate(sentences):
        # 괄호 개수 체크
        open_bracket_count += sentence.count("(")
        open_bracket_count -= sentence.count(")")

        # 문장이 20자 이하이거나, 괄호가 닫히지 않았다면 다음 문장과 결합
        if (len(sentence) <= 20 and i < len(sentences) - 1) or open_bracket_count > 0:
            temp += sentence + " "
        else:
            temp += sentence
            merged_sentences.append(temp)
            temp = ""  # temp 초기화

    if temp:  # 마지막 문장이 남아 있다면 추가
        merged_sentences.append(temp.strip())

    # 마지막 문장은 반드시 마지막에서 두 번째 문장과 결합
    if len(merged_sentences) > 1:
        merged_sentences[-2] += " " + merged_sentences[-1]
        merged_sentences.pop()  # 마지막 문장 제거 (이미 합쳐졌으므로)

    return merged_sentences

def clean_blank(text):
    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환
    return text

In [ ]:
test['cleaned_input_DEC2'] = test['input_DEC2'].apply(clean_text)

test_ = test.copy()
test_['chunked_input'] = test['cleaned_input_DEC2'].apply(split_sentences)
test_chunked = test_.explode('chunked_input')[['chunked_input']].reset_index(drop=False)

test_chunked['chunked_input'] = test_chunked['chunked_input'].apply(clean_blank)

test_chunked.drop(test_chunked[test_chunked['chunked_input'] == ''].index, inplace=True)
test_chunked.reset_index(drop=True, inplace=True)
test_chunked.to_csv(path+"/TEST_dec2_chunked.csv", index=False)

# koGemma-7b finetuning을 위한 전처리

In [ ]:
import re
import pandas as pd

def clean_text(text):
    text = str(text)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s(),.!?\'\"\[\]\{\}:]', '', text)
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)
    return text

def split_sentences(text):
    text = str(text)
    sentences = re.split(r'(?<=[.!?])\s', text)
    sentences = [s.strip() for s in sentences if s]

    merged = []
    temp = ""
    open_bracket_count = 0
    for i, sentence in enumerate(sentences):
        open_bracket_count += sentence.count("(")
        open_bracket_count -= sentence.count(")")
        if (len(sentence) <= 20 and i < len(sentences) - 1) or open_bracket_count > 0:
            temp += sentence + " "
        else:
            temp += sentence
            merged.append(temp.strip())
            temp = ""
    if temp:
        merged.append(temp.strip())
    if len(merged) > 1:
        merged[-2] = merged[-2] + " " + merged[-1]
        merged.pop()
    return merged

def clean_blank(text):
    return re.sub(r'\s+', ' ', str(text).strip())

def get_char_type(char):
    if char.isspace():
        return "space"
    elif '가' <= char <= '힣':
        return "hangul"
    elif char.isdigit():
        return "num"
    elif ('A' <= char <= 'Z') or ('a' <= char <= 'z'):
        return "eng"
    else:
        return "symbol"

def chunk_dataframe(df, input_cleaned_col, output_cleaned_col=None):
    df = df.copy()
    df['chunked_input'] = df[input_cleaned_col].apply(split_sentences)
    if output_cleaned_col:
        df['chunked_output'] = df[output_cleaned_col].apply(split_sentences)

    df_input = df.explode('chunked_input').reset_index(drop=False)
    df_input['chunked_input'] = df_input['chunked_input'].apply(clean_blank)

    if output_cleaned_col:
        df_output = df.explode('chunked_output').reset_index(drop=False)
        df_output['chunked_output'] = df_output['chunked_output'].apply(clean_blank)
        if not df_input['index'].equals(df_output['index']):
            raise ValueError("입력과 출력의 explode 결과 인덱스가 일치하지 않습니다.")
        df_input['chunked_output'] = df_output['chunked_output']

    df_input = df_input[df_input['chunked_input'] != ''].reset_index(drop=True)
    df_input['len'] = df_input['chunked_input'].apply(len)
    return df_input

def process_files(train_file, test_file, input_col, output_col=None,
                  valid_file=None,
                  cleaned_input_col="cleaned_input", cleaned_output_col="cleaned_output",
                  train_out_csv="data/train_chunked.csv", test_out_csv="data/test_chunked.csv"):

    train_df = pd.read_csv(train_file)
    if valid_file:
        valid_df = pd.read_csv(valid_file)
        train_df = pd.concat([train_df, valid_df]).reset_index(drop=True)
    test_df = pd.read_csv(test_file)

    train_df[cleaned_input_col] = train_df[input_col].apply(clean_text)
    test_df[cleaned_input_col] = test_df[input_col].apply(clean_text)
    if output_col:
        train_df[cleaned_output_col] = train_df[output_col].apply(clean_text)

    train_chunked = chunk_dataframe(train_df, cleaned_input_col, cleaned_output_col if output_col else None)
    test_chunked = chunk_dataframe(test_df, cleaned_input_col)

    train_chunked.to_csv(train_out_csv, index=False)
    test_chunked.to_csv(test_out_csv, index=False)

def process_standard():
    process_files(
        train_file=path+"/train.csv",
        test_file=path+"test.csv",
        input_col="input",
        output_col="output",
        cleaned_input_col="cleaned_input",
        cleaned_output_col="cleaned_output",
        train_out_csv="data/train_chunked.csv",
        test_out_csv="data/test_chunked.csv"
    )

def process_dec1():
    process_files(
        train_file=path+"/TRAIN_dec1.csv",
        test_file=path+"/TEST_dec1.csv",
        input_col="input_DEC1",
        output_col="output",
        valid_file=path+"valid_dec1.csv",
        cleaned_input_col="cleaned_input_DEC1",
        cleaned_output_col="cleaned_output",
        train_out_csv="data/train_chunked_dec1.csv",
        test_out_csv="data/test_chunked_dec1.csv"
    )

def filter_sentence(sentence, allowed_labels={"hangul", "eng", "space", "num"}):
    sentence = str(sentence)
    return "".join(char for char in sentence if get_char_type(char) in allowed_labels)

def filter_df(df, input_col, output_col=None, new_input_col="input", new_output_col="output"):
    df = df.copy()
    df[new_input_col] = df[input_col].apply(filter_sentence)
    if output_col:
        df[new_output_col] = df[output_col].apply(filter_sentence)
        return df[[new_input_col, new_output_col]]
    return df[[new_input_col]]

def process_finetuning(in_train_csv, in_test_csv, out_train_csv, out_test_csv):
    train = pd.read_csv(in_train_csv).drop(columns="index", errors='ignore')
    test = pd.read_csv(in_test_csv).drop(columns="index", errors='ignore')

    train_filtered = filter_df(train, "chunked_input", "chunked_output", new_input_col="input", new_output_col="output")
    test_filtered = filter_df(test, "chunked_input", output_col=None, new_input_col="input")

    train_filtered.to_csv(out_train_csv, index=False)
    test_filtered.to_csv(out_test_csv, index=False)

def process_finetuning_original():
    process_finetuning("data/train_chunked.csv", "data/test_chunked.csv",
                         "data/finetuning_train_orginal.csv", "data/finetuning_test_original.csv")

def process_finetuning_mlp():
    process_finetuning("data/train_chunked_dec1.csv", "data/test_chunked_dec1.csv",
                         "data/finetuning_train_mlp.csv", "data/finetuning_test_mlp.csv")

def finetuning_dataset_maker():
    process_standard()
    process_dec1()
    process_finetuning_original()
    process_finetuning_mlp()

if __name__ == "__main__":
    finetuning_dataset_maker()

# Seqmatcher에서 사용할 데이터 전처리

- electra 결과 문장부호 제거

In [ ]:
dec2 = pd.read_csv(path+"/TEST_dec2_chunked.csv")

In [ ]:
dec2_dict = {}

for idx, text in dec2["chunked_input"].items():
    char_list = [(char, get_char_type(char)) for char in text]
    dec2_dict[idx] = pd.DataFrame(char_list, columns=["Character", "Label"])

# 허용할 라벨 리스트
allowed_labels = {"hangul", "eng", "space", "num"}

# 각 row별로 문자 필터링 후 합친 문장을 저장

cleaned_dec2 = pd.DataFrame({
    "chunked_input": [
        "".join(dec2_dict[idx].loc[dec2_dict[idx]["Label"].isin(allowed_labels), "Character"])
        for idx in dec2_dict
    ]
})

# cc: chunked & cleaned
# index는 지움
dec2_cc = cleaned_dec2.rename(columns={"chunked_input": "input"})

In [ ]:
# chunking시 선공백처리, 후공백처리 (clean_blank)에 따라 문장 분리가 안일어나는 문장 하나가 있음...
# 장점 접근성이 나쁘지 않습니다 / 부산역에서 버스 15분 로비가 3층이라 호텔 출입 시 프라이빗 보장이 됩니다
# 분리시켜서 최종적으로 4808행을 4809행으로 늘림

# 기존 데이터 가져오기
data = dec2_cc.copy()

# 104번째 행의 값 가져오기
original_text = data.iloc[104, 0]

# "장점 접근성이 나쁘지 않습니다"를 기준으로 분리
split_point = "장점 접근성이 나쁘지 않습니다"
if split_point in original_text:
    first_part = split_point
    second_part = original_text.replace(split_point, "").strip()  # 나머지 부분 가져오기

    # 104번째 행 업데이트
    data.iloc[104, 0] = first_part

    # 이후 행들을 한 칸씩 밀기
    new_rows = pd.DataFrame({0: [second_part]})  # 새로운 행 생성
    data = pd.concat([data.iloc[:105], new_rows, data.iloc[105:]]).reset_index(drop=True)

data.iloc[105, 0] = data.iloc[105, 1]
dec2_cc = data.drop(columns=0)

In [ ]:
dec2_cc.to_csv(path+"/seqmatcher_ELECTRA.csv", index=False) # seqmatcher에서 사용할 dec2 청크 문장부호 제거입니다.

- 원본 데이터 청크와 문장부호 제거

In [ ]:
original = pd.read_csv(path+"/test.csv")

In [ ]:
original["cleaned_input"] = original["input"].apply(clean_text)
original_ = original.copy()
original_['chunked_input'] = original['cleaned_input'].apply(split_sentences)
original_chunked_ = original_.explode('chunked_input')[['chunked_input']].reset_index(drop=False)
original_chunked_['chunked_input'] = original_chunked_['chunked_input'].apply(clean_blank)

original_chunked_.drop(original_chunked_[original_chunked_['chunked_input'] == ''].index, inplace=True)
original_chunked_.reset_index(drop=True, inplace=True)
original_chunked_['len'] = original_chunked_['chunked_input'].apply(lambda x: len(x))

In [ ]:
original_dict = {}

for idx, text in original_chunked_["chunked_input"].items():
    char_list = [(char, get_char_type(char)) for char in text]
    original_dict[idx] = pd.DataFrame(char_list, columns=["Character", "Label"])

allowed_labels = {"hangul", "eng", "space", "num"}

# 각 row별로 문자 필터링 후 합친 문장을 저장

cleaned_original = pd.DataFrame({
    "chunked_input": [
        "".join(original_dict[idx].loc[original_dict[idx]["Label"].isin(allowed_labels), "Character"])
        for idx in original_dict
    ]
})

In [ ]:
original_cc = cleaned_original.rename(columns={"chunked_input": "input"})

In [ ]:
original_cc.to_csv(path+"/seqmatcher_ORIGINAL.csv", index=False)

# 2차 해독_finetuning

Finetuning은 Runpod 환경에서 실행됩니다.

* OS: ubuntu22.04
* CPU: Intel(R) Xeon(R) Platinum 8462Y 16 Core CPU
* RAM: 251GB
* GPU: A100 SXM
* VRAM: 80GB
* py3.10, pytorch 2.2.0, cuda12.1.1 <br>
깃허브 링크 <br>
https://github.com/junseok0913/Noised-Korean-Review-Deobfuscator

In [ ]:
import os
import random
import pandas as pd
from tqdm.auto import tqdm
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    TrainingArguments
)

from datasets import Dataset
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

In [ ]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# cuDNN 관련 설정
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"  # (PyTorch 1.8 이상에서 추천)
torch.use_deterministic_algorithms(True)

os.environ["HF_HOME"] = path+"/workspace/huggingface"
os.environ["TRANSFORMERS_CACHE"] = path+"/workspace/huggingface"

In [ ]:
# 통합 파이프라인 함수
def full_training_pipeline(train_df, output_dir, base_model_path, adapter_model_path, epoch):
    # 8-bit BitsAndBytes config (8-bit uses bfloat16)
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

    model_id = 'beomi/gemma-ko-7b'

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map={"": 0},
        cache_dir="/workspace/huggingface"    # <--- volume 지정
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        cache_dir="/workspace/huggingface"    # <--- volume 지정
    )

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'right'

    train = Dataset.from_pandas(train_df)

    def prompting(input, output):
        prompt = (
            "<start_of_turn> Your task is to transform the given obfuscated Korean review into a clear, correct, and natural-sounding Korean review that reflects its original meaning. The number of words and letters per word must be observed.\n"
            f"Input: {input}\n"
            "<end_of_turn>\n"
            "<start_of_turn>Assistant:\n"
            f"Output: {output}"
        )
        return prompt

    def chat_format(row):
        prompt = prompting(row["input"], row["output"])
        tokens = tokenizer.encode(prompt, truncation=True, max_length=512)
        row["input_ids"] = tokens
        return row

    train = train.map(chat_format, batched=False, num_proc=4)

    # LoRA 설정
    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=[
            "q_proj", "v_proj", "k_proj", "o_proj",
            "gate_proj", "down_proj", "up_proj"
        ],
        lora_dropout=0.1,
        bias='none',
        task_type='CAUSAL_LM'
    )

    model = get_peft_model(model, lora_config)
    model.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    training_args = TrainingArguments(
        seed=42,
        output_dir=output_dir,
        num_train_epochs=epoch,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=8,
        optim="paged_adamw_32bit",
        eval_strategy="no",
        logging_dir="./logs",
        logging_steps=50,
        warmup_steps=20,
        logging_strategy="steps",
        learning_rate=2e-4,
        group_by_length=True,
        save_strategy="epoch",
        fp16=True
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=train,
        args=training_args,
        peft_config=lora_config,
        formatting_func=lambda x: x['input_ids']
    )

    trainer.train()

    # 모델 저장 및 로드
    trainer.model.save_pretrained(adapter_model_path)

    model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    torch_dtype=torch.float16,
    cache_dir=path+"/workspace/huggingface")

    model = PeftModel.from_pretrained(
        model,
        adapter_model_path,
        device_map='auto',
        torch_dtype=torch.float16
    )
    return model

if __name__ == "__main__":

    #파일 위치는 실행 환경에 따라 바꿔주세요.

    #2차 해독_finetuning_ver1(input:MLP)
    train0 = pd.read_csv(path+'/finetuning_train_mlp.csv')
    output_dir0=path+"/results0221"
    base_model_path0 = "beomi/gemma-ko-7b"
    adapter_model_path0 =  path+"/workspace/results0221/lora-adapter-epoch3"

    model0 = full_training_pipeline(train0, output_dir0, base_model_path0, adapter_model_path0, epoch=3)
    model0.save_pretrained(path+"/workspace/results0221/gemma-finetuning-epoch3")
    print("2차 해독_finetuning_ver1(input:MLP) 완료")

    #2차 해독_finetuning_ver1(input:원본)
    train1 = pd.read_csv(path+'finetuning_train_original.csv')
    output_dir1=path+"/results0225"
    base_model_path1 = "beomi/gemma-ko-7b"
    adapter_model_path1 = path+"/workspace/results0225/lora-adapter-epoch5"

    model1 = full_training_pipeline(train1, output_dir1, base_model_path1, adapter_model_path1, epoch=5)
    model1.save_pretrained(path+"/workspace/results0225/gemma-finetuning-epoch5")
    print("2차 해독_finetuning_ver1(input:원본) 완료")

In [ ]:
def restore_reviews(FINETUNE_MODEL, BASE_MODEL, test):
    finetune_model = AutoModelForCausalLM.from_pretrained(
        FINETUNE_MODEL, cache_dir="/workspace/huggingface", device_map={"": 0}
    )
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir="/workspace/huggingface")

    text_gen_pipeline = pipeline(
        "text-generation",
        model=finetune_model,
        tokenizer=tokenizer,
    )

    restored_reviews = []
    total_reviews = len(test)

    with tqdm(total=total_reviews, desc="Processing", unit="review") as pbar:
        for index, row in test.iterrows():
            query = row['input']
            prompt = (
                "<start_of_turn> Your task is to transform the given obfuscated Korean review into a clear, correct, and natural-sounding Korean review that reflects its original meaning. The number of words and letters per word must be observed.\n"
                f"Input: {query}\n"
                "<end_of_turn>\n"
                "<start_of_turn>Assistant:\n"
                "Output:"
            )

            generated = text_gen_pipeline(
                prompt,
                num_return_sequences=1,
                temperature=0.2,
                top_p=0.9,
                max_new_tokens=len(query),
                do_sample=True,
                eos_token_id=tokenizer.eos_token_id
            )

            generated_text = generated[0]['generated_text']
            output_start = generated_text.find("Output:")

            if output_start != -1:
                restored_reviews.append(generated_text[output_start + len("Output:"):].strip())
            else:
                restored_reviews.append(generated_text.strip())

            pbar.update(1)

    return restored_reviews


if __name__ == "__main__":

  #파일 위치는 실행 환경에 따라 바꿔주세요.

  FINETUNE_MODEL0 = path+"/workspace/results0221/gemma-finetuning-epoch3"
  BASE_MODEL0 = "beomi/gemma-ko-7b"
  test0 = pd.read_csv(path+'/finetuning_test_mlp.csv')
  restored_reviews0 = restore_reviews(FINETUNE_MODEL0, BASE_MODEL0, test0)

  FINETUNE_MODEL1 = path+"/workspace/results0225/2gemma-finetuning-epoch5"
  BASE_MODEL1 = "beomi/gemma-ko-7b"
  test1 = pd.read_csv(path+'/finetuning_test_original.csv')
  restored_reviews1 = restore_reviews(FINETUNE_MODEL1, BASE_MODEL1, test1)

In [ ]:
submission = pd.read_csv(path+"/sample_submission.csv")
submission = pd.DataFrame()
submission['output'] = restored_reviews0
submission['output'] = submission['output'].apply(lambda x: x.split("<end_of_turn>")[0])
submission.to_csv(path+'/kogemma_0222_raw.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
submission = pd.read_csv(path+"/sample_submission.csv")
submission = pd.DataFrame()
submission['output'] = restored_reviews1
submission['output'] = submission['output'].apply(lambda x: x.split("<end_of_turn>")[0])
submission.to_csv(path+'/kogemma_0226_raw.csv', index = False, encoding = 'utf-8-sig')

# Sequence Matcher

In [ ]:
import pandas as pd
import difflib
from jamo import h2j
from rapidfuzz import fuzz
import re

## koGemma 후처리 함수

In [ ]:
def fuzzy_equal(a: str, b: str, threshold: float) -> bool:
    if len(a) != len(b):
        return False
    return fuzz.ratio(h2j(a), h2j(b)) >= threshold

def refine_replace_block(tokens_a, tokens_b, threshold: float):
    n, m = len(tokens_a), len(tokens_b)
    dp = [[0]*(m+1) for _ in range(n+1)]
    for i in range(n):
        for j in range(m):
            if fuzzy_equal(tokens_a[i], tokens_b[j], threshold):
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i+1][j], dp[i][j+1])
    i, j = n, m
    matched = []
    while i > 0 and j > 0:
        if fuzzy_equal(tokens_a[i-1], tokens_b[j-1], threshold):
            matched.append((i-1, j-1))
            i -= 1
            j -= 1
        else:
            if dp[i-1][j] >= dp[i][j-1]:
                i -= 1
            else:
                j -= 1
    matched.reverse()

    opcodes = []
    a_start, b_start = 0, 0
    for a_idx, b_idx in matched:
        if a_start < a_idx:
            opcodes.append(('delete', a_start, a_idx, b_start, b_start))
        opcodes.append(('equal', a_idx, a_idx+1, b_idx, b_idx+1))
        a_start = a_idx + 1
        b_start = b_idx + 1
    if a_start < n:
        opcodes.append(('delete', a_start, n, b_start, b_start))
    return opcodes

def process_pairwise_block(block_org, block_a, block_b, similarity_threshold):
    """
    동일 길이의 토큰 블록에 대해, 원본(block_org)과의 유사도를 비교하여
    더 유사한 토큰을 선택합니다.
    """
    result = []
    for org_tok, a_tok, b_tok in zip(block_org, block_a, block_b):
        if len(a_tok) == len(b_tok):
            score_a = fuzz.ratio(h2j(org_tok), h2j(a_tok))
            score_b = fuzz.ratio(h2j(org_tok), h2j(b_tok))
            result.append(a_tok if score_a > score_b else b_tok)
        else:
            result.append(a_tok)
    return result

def process_replace_block(block_org, block_a, block_b, similarity_threshold):
    """
    replace 구간 처리:
      - 토큰 수가 같으면 pairwise 비교,
      - 다르면 LCS 기반으로 재세분화(refine_replace_block)를 수행하여 세부 opcodes에 따라 처리.
    """
    if len(block_a) == len(block_b):
        return process_pairwise_block(block_org, block_a, block_b, similarity_threshold)
    else:
        refined_opcodes = refine_replace_block(block_a, block_b, similarity_threshold)
        result = []
        for tag, i1, i2, j1, j2 in refined_opcodes:
            if tag == 'equal':
                sub_org = block_org[i1:i2]
                sub_a = block_a[i1:i2]
                sub_b = block_b[j1:j2]
                if len(sub_a) == len(sub_b):
                    result.extend(process_pairwise_block(sub_org, sub_a, sub_b, similarity_threshold))
                else:
                    result.extend(sub_b)
            elif tag == 'delete':
                result.extend(block_a[i1:i2])
            elif tag == 'replace':
                result.extend(block_b[j1:j2])
        return result

def transform_sentence(original: str, sent_a: str, sent_b: str, similarity_threshold=50) -> str:
    tokens_org = original.split()
    tokens_a = sent_a.split()
    tokens_b = sent_b.split()

    matcher = difflib.SequenceMatcher(None, tokens_a, tokens_b)
    opcodes = matcher.get_opcodes()

    result_tokens = []
    for tag, i1, i2, j1, j2 in opcodes:
        if tag == 'equal':
            result_tokens.extend(tokens_b[j1:j2])
        elif tag == 'delete':
            result_tokens.extend(tokens_a[i1:i2])
        elif tag == 'insert':
            continue  # insert 구간은 무시
        elif tag == 'replace':
            block_org = tokens_org[i1:i2]
            block_a = tokens_a[i1:i2]
            block_b = tokens_b[j1:j2]
            result_tokens.extend(process_replace_block(block_org, block_a, block_b, similarity_threshold))

    len_diff = len(tokens_a) - len(result_tokens)
    if len_diff > 0:
        result_tokens.extend(tokens_a[-len_diff:])
    elif len_diff < 0:
        result_tokens = result_tokens[:len(tokens_a)]

    return " ".join(result_tokens)

if __name__ == "__main__":

    original = ("특힌 캑쉴 1012횬눈 쌈뮷쉴 뒷펀 예열켠 싫욉귀갸 샴얼 죵때료 토엷한 샥먁한 푸갸 쉬야위 절뱌늚 쨔찌합뉘답")
    sent_a = ("특히 객실 1012호는 사무실 뒤편 에어컨 실외기가 샤워 초대로 도염한 싼막한 뷰가 시양이 절반을 차취합니다")
    sent_b = ("특히 객실 1012호는 사무실 뒤편 에어컨 실외기가 샤워 창틀 쪽으로 돌출한 삭막한 뷰가 시야를 차지합니다")

    # original = ("3밗 옌아켓탁갸 1빡 학꼲 딴 뗄룟 옳걷닸")
    # sent_a = ("3박 예약했다가 1박 하고 딴 데로 옮겼다")
    # sent_b = ("3박 예약했다가 1박 하고 다 데로 돌려 받았다")

    original =  ("쮜겡군 끄 와츙웨 왠 네 물켱는 쥐엊치꽃 냐걋눈 견짐 앍코 본뉘 뀔를 팡 앉에 놓꼬 낚왓단는 쮜껙긔 만뤠 샵창이 먁슴떠끼룰 줆엊터락굡욤")
    sent_a =    ("치객은 그 와중에 왜 내 물거는 찢어지고 나가는 건지 않고 보니 키를 방 안에 놓고 나왔다는 취객의 만에 사장이 막스더기를 주셨더라고요")
    sent_b =    ("치킨은 그 와중에 왜 내 물컵은 치워지고 나갔는 건지 알고 보니 키를 방 안에 놓고 나왔다는 치킨의 말에 사장이 마스터키를 주었다고요")

    merged = transform_sentence(original, sent_a, sent_b, similarity_threshold=50)
    print("[최종 결과]\n", merged)
    print("[문장 A 토큰 수] ", len(sent_a.split()))
    print("[최종 결과 토큰 수]", len(merged.split()))


## electra 잘못된 문장부호 추론 대체

In [ ]:
'''
TEST_0328
kogemma raw
베개에 고무의처럼 얼룩이 있어서 그린 줄 알았는데 커버를 살짝 벗겨보니 누가 침을 흘렸던 베개인지 오염돼서 생긴 얼룩에 커버를 씌운 거였습니다
electra
베개에 교문의처럼 얼룩이 있어서 그림일 줄 알았는데, 거버를 살짝 벗겨보니 누가 짐을 흐렸던 베개인지 옵염돼서 생긴 얼룩게 커버를 !은 것이었습니다.
후처리
베개에 고무의처럼 얼룩이 있어서 그림일 줄 알았는데 커버를 살짝 벗겨보니 누가 침을 흘렸던 베개인지 오염돼서 생긴 얼룩에 커버를 은 것이었습니다
원본
볘캐웨 꽃뮨늬철렴 얽륙익 잊엿셔 끓림민 출 앍얏눈뒈, 켜벌를 삵짝 벋겊봉뉘 눌갉 찜울 흙렸뎐 뻬게윈쥐 옮엄퉤써 쌩낀 열류궤 컵뻘를 씔운 겸씩얻숩뉜타.

electra의 경우 한글 글자를 문장기호로 추론하는 경우가 있음.
'''

In [ ]:
electra = pd.read_csv(path+"/seqmatcher_ELECTRA.csv")
test = pd.read_csv(path+"/finetuning_TEST_mlp.csv")

In [ ]:
# 공백 기준으로 어절을 나누는 함수
def split_sentence(sentence):
    return sentence.split()

# 어절 길이 비교 후 대체하는 함수
def correct_sentence(electra_sentence, test_sentence):
    electra_words = split_sentence(electra_sentence)
    test_words = split_sentence(test_sentence)

    # 최소한의 길이로 맞춰 비교
    min_len = min(len(electra_words), len(test_words))

    # 어절 길이가 다르면 test의 어절로 교체
    corrected_words = [
        test_words[i] if len(electra_words[i]) != len(test_words[i]) else electra_words[i]
        for i in range(min_len)
    ]

    # 만약 electra 문장이 짧으면 test 문장 어절 추가
    if len(electra_words) < len(test_words):
        corrected_words.extend(test_words[min_len:])

    return " ".join(corrected_words)

# Electra 문장 수정 적용
electra["no_more_nueggimpyo"] = [
    correct_sentence(e_sent, t_sent) for e_sent, t_sent in zip(electra["input"], test["input"])
]

In [ ]:
electra["no_more_nueggimpyo"].to_csv(path+"/seqmatcher2_ELECTRA.csv", index=False)

## kogemma 추론의 ZWS 공백문자 제거

### kogemma v1 (mlp)

In [ ]:
import pandas as pd

In [ ]:
# 파일 경로
file_path = path+"/kogemma_0222_raw.csv"

# CSV 파일 로드 (모든 열을 문자열로 로드)
zws = pd.read_csv(file_path, dtype=str)

# Zero-Width Space (ZWS, U+200B) 포함 여부 확인
zws_mask = zws.applymap(lambda x: "\u200b" in str(x) if pd.notna(x) else False)

# ZWS가 포함된 셀들의 위치 찾기
zws_cells = [(row, col) for row, col in zip(*zws_mask.to_numpy().nonzero())]
zws_count = len(zws_cells)  # ZWS 포함된 셀 개수
print(zws_count)


In [ ]:
# 각 셀에서 ZWS 위치를 저장할 새로운 컬럼 추가
zws["ZWS_positions"] = zws.applymap(lambda x: [i for i, char in enumerate(str(x)) if char == "\u200b"] if pd.notna(x) else [])

# ZWS 제거 함수 정의
def remove_zws(cell):
    if isinstance(cell, str):  # 문자열인 경우에만 처리
        return cell.replace("\u200b", "")
    return cell  # 그 외는 그대로 반환

# ZWS 제거 적용
zws_cleaned = zws.drop(columns=["ZWS_positions"]).applymap(remove_zws)

# 새로운 파일로 저장
cleaned_file_path = path+"/kogemma_0222_zws.csv"
zws_cleaned.to_csv(cleaned_file_path, index=False, encoding="utf-8-sig")

### kogemma v2 (원본)

In [ ]:
# 파일 경로
file_path = path+"/kogemma_0226_raw.csv

# CSV 파일 로드 (모든 열을 문자열로 로드)
zws = pd.read_csv(file_path, dtype=str)

# Zero-Width Space (ZWS, U+200B) 포함 여부 확인
zws_mask = zws.applymap(lambda x: "\u200b" in str(x) if pd.notna(x) else False)

# ZWS가 포함된 셀들의 위치 찾기
zws_cells = [(row, col) for row, col in zip(*zws_mask.to_numpy().nonzero())]
zws_count = len(zws_cells)  # ZWS 포함된 셀 개수
print(zws_count)


In [ ]:
# 각 셀에서 ZWS 위치를 저장할 새로운 컬럼 추가
zws["ZWS_positions"] = zws.applymap(lambda x: [i for i, char in enumerate(str(x)) if char == "\u200b"] if pd.notna(x) else [])

# ZWS 제거 함수 정의
def remove_zws(cell):
    if isinstance(cell, str):  # 문자열인 경우에만 처리
        return cell.replace("\u200b", "")
    return cell  # 그 외는 그대로 반환

# ZWS 제거 적용
zws_cleaned = zws.drop(columns=["ZWS_positions"]).applymap(remove_zws)

# 새로운 파일로 저장
cleaned_file_path = path+"/kogemma_0226_zws.csv"
zws_cleaned.to_csv(cleaned_file_path, index=False, encoding="utf-8-sig")

# 데이터 프레임 가공

### kogemma v1 (mlp)

In [ ]:
# 데이터 로드
original = pd.read_csv(path+"/seqmatcher_ORIGINAL.csv")
electra = pd.read_csv(path+"/seqmatcher2_ELECTRA.csv")
electra = electra.rename(columns={"no_more_nueggimpyo": "input"})
kogemma = pd.read_csv(path+"/kogemma_0222_zws.csv").rename(columns={"output": "input"})

In [ ]:
# 새로운 데이터프레임 생성
df = pd.DataFrame()
for column in electra.columns:
    sent_original_list = original[column].astype(str).tolist()
    sent_a_list = electra[column].astype(str).tolist()
    sent_b_list = kogemma[column].astype(str).tolist()
    df[f"{column}_electra"] = sent_a_list
    df[f"{column}_kogemma"] = [transform_sentence(o ,a, b, 50) for o, a, b in zip(sent_original_list, sent_a_list, sent_b_list)]

# 결과 저장
df.to_csv(path+"/0227_electra_vs_kogemmaV1.csv", index=False)

In [ ]:
index_data = pd.read_csv(path+"/test_chunked_dec1.csv")
df = pd.read_csv(path+"/0227_electra_vs_kogemmaV1.csv")
df["index"] = index_data["index"]
df_unchunk = df.groupby("index")["input_kogemma"].apply(lambda x: " ".join(x)).reset_index()
df_unchunk = df_unchunk.drop(columns="index")
df_unchunk.to_csv(path+"/0227_unchunk_electra_vs_kogemmaV1.csv", index=False)

### kogemma v2 (원본)

In [ ]:
# 데이터 로드
original = pd.read_csv(path+"/seqmatcher_ORIGINAL.csv")
electra = pd.read_csv(path+"/seqmatcher2_ELECTRA.csv")
electra = electra.rename(columns={"no_more_nueggimpyo": "input"})
#kogemma = pd.read_csv("data/seqmatcher_kogemma.csv")
kogemma = pd.read_csv(path+"/kogemma_0226_zws.csv").rename(columns={"output": "input"})

In [ ]:
# 새로운 데이터프레임 생성
df = pd.DataFrame()
for column in electra.columns:
    sent_original_list = original[column].astype(str).tolist()
    sent_a_list = electra[column].astype(str).tolist()
    sent_b_list = kogemma[column].astype(str).tolist()
    df[f"{column}_electra"] = sent_a_list
    df[f"{column}_kogemma"] = [transform_sentence(o ,a, b, 50) for o, a, b in zip(sent_original_list, sent_a_list, sent_b_list)]

# 결과 저장
df.to_csv(path+"/0227_electra_vs_kogemmaV2.csv", index=False)

In [ ]:
index_data = pd.read_csv(path+"/test_chunked_dec1.csv")
df = pd.read_csv(path+"/0227_electra_vs_kogemmaV2.csv")
df["index"] = index_data["index"]
df_unchunk = df.groupby("index")["input_kogemma"].apply(lambda x: " ".join(x)).reset_index()
df_unchunk = df_unchunk.drop(columns="index")
df_unchunk.to_csv(path+"/0227_unchunk_electra_vs_kogemmaV2.csv", index=False)

# 문장 부호 붙이기

In [ ]:
def replace_with_kogemma(test_text, kogemma_text):
    """
    test_text에서 (가-힣, a-z, A-Z, 0-9)만 순회하며
    kogemma_text(공백 제거) 글자들을 순서대로 치환.
    만약 글자 수가 다르면 알림 메시지를 리턴.
    """
    # 1) kogemma_text에서 공백 제거 -> 각 글자 리스트
    kogemma_chars = list(kogemma_text.replace(" ", ""))

    # 2) test_text 내 치환 대상 글자 수 세기
    test_valid_chars = re.findall(r'[가-힣a-zA-Z0-9]', test_text)
    test_valid_count = len(test_valid_chars)
    kogemma_count = len(kogemma_chars)

    # 3) 글자 수가 다르면 알림만 리턴
    #if test_valid_count != kogemma_count:
    #   return f"[글자 수 불일치] test:{test_valid_count}, unchunked:{kogemma_count} - 내용: {test_text}"

    # 4) 한 글자씩 교체 진행
    result_chars = []
    idx = 0  # kogemma_chars 인덱스
    for ch in test_text:
        # (가-힣, a-zA-Z, 0-9)이면 교체
        if re.match(r'[가-힣a-zA-Z0-9]', ch):
            result_chars.append(kogemma_chars[idx])
            idx += 1
        else:
            # 그 외 (공백, 문장부호, 이모티콘 등)은 그대로 둠
            result_chars.append(ch)

    return "".join(result_chars)

### kogemma v1 (mlp)

In [ ]:
test_df = pd.read_csv(path+"/test.csv")
kogemma_unchunked_df = pd.read_csv(path+"/0227_unchunk_electra_vs_kogemmaV1.csv")

In [ ]:
# DF를 순회하면서 치환 결과를 새로운 컬럼으로 넣음
df_result = pd.DataFrame(columns=['input', 'input_kogemma', 'output'])

for i in range(len(test_df)):
    t = test_df.loc[i, 'input']
    k = kogemma_unchunked_df.loc[i, 'input_kogemma']
    replaced_text = replace_with_kogemma(t, k)

    df_result.loc[i] = [t, k, replaced_text]

In [ ]:
sample = pd.read_csv(path+"/sample_submission.csv")
sample["output"] = df_result["output"]
sample.to_csv(path+"/0227_koGemma_v1.csv", index=False)
temp = pd.read_csv(path+"/test.csv")
sum(len(temp["input"].iloc[i]) != len(sample["output"].iloc[i]) for i in range(len(temp))) # 길이 검증, 0 나와야함

### kogemma v2 (원본)

In [ ]:
test_df = pd.read_csv(path+"/test.csv")
kogemma_unchunked_df = pd.read_csv("data/0227_unchunk_electra_vs_kogemmaV2.csv")

In [ ]:
# DF를 순회하면서 치환 결과를 새로운 컬럼으로 넣음
df_result = pd.DataFrame(columns=['input', 'input_kogemma', 'output'])

for i in range(len(test_df)):
    t = test_df.loc[i, 'input']
    k = kogemma_unchunked_df.loc[i, 'input_kogemma']
    replaced_text = replace_with_kogemma(t, k)

    df_result.loc[i] = [t, k, replaced_text]

In [ ]:
sample = pd.read_csv(path+"/sample_submission.csv")
sample["output"] = df_result["output"]
sample.to_csv(path+"/0227_koGemma_v2.csv", index=False)
temp = pd.read_csv("test.csv")
sum(len(temp["input"].iloc[i]) != len(sample["output"].iloc[i]) for i in range(len(temp))) # 길이 검증, 0 나와야함

# Sentence Selection with perplexity computed by Bllossom 8B

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def calculate_perplexity(sentence: str, model, tokenizer, device: str) -> float:
    encodings = tokenizer(sentence, return_tensors="pt")
    input_ids = encodings.input_ids.to(device)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss

    # perplexity
    perplexity = torch.exp(loss)
    return perplexity.item()

In [ ]:
model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [ ]:
data1 = pd.read_csv(path+"/0227_koGemma_v1.csv")
data2 = pd.read_csv(path+"/submission_electra.csv")
data3 = pd.read_csv(path+"/0227_koGemma_v2.csv")

## Cleaning Data

In [ ]:
import re
def clean_text(text):
    text = re.sub(r'[^가-힣a-zA-Z0-9\s(),.!?\'\"\[\]\{\}:]', '', text)
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

def split_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s', text)
    sentences = [sentence.strip() for sentence in sentences if sentence]
    merged_sentences = []
    temp = ""
    open_bracket_count = 0

    for i, sentence in enumerate(sentences):
        open_bracket_count += sentence.count("(")
        open_bracket_count -= sentence.count(")")
        if (len(sentence) <= 15 and i < len(sentences) - 1) or open_bracket_count > 0:
            temp += sentence + " "
        else:
            temp += sentence
            merged_sentences.append(temp)
            temp = ""

    if temp:
        merged_sentences.append(temp.strip())

    if len(merged_sentences) > 1:
        merged_sentences[-2] += " " + merged_sentences[-1]
        merged_sentences.pop()
    return merged_sentences


def clean_and_chunk_data(data):
    data['cleaned_output'] = data['output'].apply(clean_text)
    data['length'] = data['cleaned_output'].apply(len)
    max_length = data['length'].max()
    data['chunked_output'] = data['cleaned_output'].apply(split_sentences)
    data_chunked = data.explode('chunked_output')[['chunked_output']].reset_index(drop=False)
    data_chunked['len'] = data_chunked['chunked_output'].apply(len)
    data_chunked.drop(columns=['len'], inplace=True, errors='ignore')

    return data_chunked, max_length

data1_chunked, max_length1 = clean_and_chunk_data(data1)
data2_chunked, max_length2 = clean_and_chunk_data(data2)
data3_chunked, max_length3 = clean_and_chunk_data(data3)


In [ ]:
max_length1, max_length2, max_length3

In [ ]:
data = pd.concat([data1_chunked, data2_chunked[['chunked_output']], data3_chunked[['chunked_output']]], axis =1)
data.columns = ['index','kogemma','electra', 'kogemma_v2']
data.head()

In [ ]:
data.to_csv(path+"/data_chunked.csv", index=False)

# Kogemma vs Electra

## Candidates

In [ ]:
data = pd.read_csv(path+"/data_chunked.csv")

In [ ]:
from itertools import product

def generate_candidate_sentences(sentence1: str, sentence2: str) -> list:

    tokens1 = sentence1.split()
    tokens2 = sentence2.split()

    if len(tokens1) != len(tokens2):
        raise ValueError("두 문장의 어절 수가 동일해야 합니다.")

    candidate_tokens = []
    for t1, t2 in zip(tokens1, tokens2):
        # 어절이 동일하면 한 가지 선택지만, 다르면 두 가지 선택지를 사용합니다.
        candidate_tokens.append([t1] if t1 == t2 else [t1, t2])

    # Cartesian product를 사용해 모든 조합의 문장을 생성합니다.
    candidates = [" ".join(tokens) for tokens in product(*candidate_tokens)]
    return candidates

data['candidate'] = data.apply(lambda row: generate_candidate_sentences(row['kogemma'], row['electra']), axis=1)
data['len'] = data.apply(lambda x: len(x['candidate']) if len(x['candidate']) > 1 else 0, axis = 1)
data['sent_len'] = data.apply(lambda x: len(x['kogemma'].split()), axis = 1)

In [ ]:
data['len'].value_counts()

## perplexity

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
def clean_blank(text):
    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환
    return text

def restore_original_text(original_text, decoded_text):

    restored_text = ""
    decoded_idx = 0  # 해독된 텍스트에서 현재 참조 중인 문자 위치

    # 한글, 영어, 숫자, 공백, 문장부호만 허용하는 정규식
    valid_char_pattern = re.compile(r'[가-힣a-zA-Z0-9(),.!?\'\"\[\]\{\}:]')


    for char in original_text:
        if valid_char_pattern.match(char):
            # 유효한 문자일 경우, 해독된 문자로 대체
            if decoded_idx < len(decoded_text):
                restored_text += decoded_text[decoded_idx]
                decoded_idx += 1
            else:
                # 만약 해독된 텍스트가 끝났다면 원문 문자 그대로 추가
                restored_text += char
        else:
            # 특수문자 및 불완전한 한글은 원문 그대로 유지
            restored_text += char

    return restored_text

def check_double_blanks(text):
    return bool(re.search(r' {2,}', text))

def check_repeated_last_char(text):
    return bool(re.search(r'([가-힣])\1+$', text))

def remove_trailing_unks(text):

    cleaned_text = re.sub(r'(<unk>\s*)+$', '', text)
    return cleaned_text

def post_processing(data, data_chunked, result, test_flg = True):
  temp = data.copy()
  temp_chunked = data_chunked.copy()

  temp_chunked['result'] = result
  data_recovered = temp_chunked.groupby("index").agg(list).reset_index(drop=True)
  if test_flg:
    data_recovered.columns = ['kogemma','electra', 'result']
    data_recovered['result'] = data_recovered['result'].apply(lambda x: ' '.join(x))
  else:
    data_recovered.columns = ['input_DEC1','output','input_DEC2']
    data_recovered['input_DEC2'] = data_recovered['input_DEC2'].apply(lambda x: ' '.join(x))

  temp['input'] = temp.apply(lambda x: clean_blank(x['input']), axis = 1)

  temp['result'] = data_recovered['result']
  temp['result'] = temp.apply(lambda x: clean_blank(x['result']), axis = 1)
  temp['result'] = temp.apply(lambda x: x['result'].replace(" ",""), axis= 1)

  temp['result'] = temp.apply(lambda x: restore_original_text(x['input'],x['result']), axis = 1)

  return temp

## comparison

In [ ]:
def select_best_candidate(candidates: list, model, tokenizer, device: str, sentence1: str, sentence2: str) -> str:
    """
    후보 문장 리스트에서, 후보 수에 따라 다음과 같이 선택.
      - 후보가 1개이면: 바로 그 문장을 선택 (펄플렉시티 계산 없이)
      - 후보가 200개 이상이면: 두 원본 문장(sentence1, sentence2)만 펄플렉시티를 계산하여 더 낮은 값을 가진 문장을 선택
      - 그 외의 경우: 모든 후보 문장에 대해 펄플렉시티를 계산한 후, 가장 낮은 값을 가진 문장을 선택

    Returns:
        (best_candidate, best_ppl): 선택된 문장과 그 perplexity 값 (계산하지 않은 경우 None)
    """
    if len(candidates) == 1:
        # 후보가 단 1개이면, 바로 반환 (펄플렉시티 계산 없이)
        best_candidate = candidates[0]
        return best_candidate

    elif len(candidates) >= 200:
        # 후보가 너무 많으면, 원본 두 문장끼리만 펄플렉시티를 계산.
        ppl1 = calculate_perplexity(sentence1, model, tokenizer, device)
        ppl2 = calculate_perplexity(sentence2, model, tokenizer, device)
        best_candidate = sentence1 if ppl1 < ppl2 else sentence2
        return best_candidate

    else:
        # 후보가 200개 미만인 경우: 모든 후보에 대해 펄플렉시티를 계산.
        best_candidate = None
        best_ppl = float("inf")
        for candidate in candidates:
            ppl = calculate_perplexity(candidate, model, tokenizer, device)
            if ppl < best_ppl:
                best_ppl = ppl
                best_candidate = candidate
        return best_candidate

In [ ]:
model.eval()
from tqdm import tqdm
tqdm.pandas()
data['best_sentence'] = data.progress_apply(lambda x: select_best_candidate(x['candidate'],
                                                                            model, tokenizer, device,
                                                                            x['kogemma'],
                                                                            x['electra']), axis=1)

In [ ]:
test = pd.read_csv(path+'/test.csv')
sub = post_processing(test, data[['index','kogemma','electra']], data['best_sentence'])
sub.drop('input', axis = 1, inplace = True)
sub.columns = ['ID', 'output']
sub.to_csv(path+'/kogemma vs electra(submission).csv', index = False)

# Kogemma vs Kogemma_v2

## Candidates

In [ ]:
data = pd.read_csv(path+"/data_chunked.csv")

In [ ]:
from itertools import product

def generate_candidate_sentences(sentence1: str, sentence2: str) -> list:

    tokens1 = sentence1.split()
    tokens2 = sentence2.split()

    if len(tokens1) != len(tokens2):
        raise ValueError("두 문장의 어절 수가 동일해야 합니다.")

    candidate_tokens = []
    for t1, t2 in zip(tokens1, tokens2):
        # 어절이 동일하면 한 가지 선택지만, 다르면 두 가지 선택지를 사용합니다.
        candidate_tokens.append([t1] if t1 == t2 else [t1, t2])

    # Cartesian product를 사용해 모든 조합의 문장을 생성합니다.
    candidates = [" ".join(tokens) for tokens in product(*candidate_tokens)]
    return candidates

data['candidate'] = data.apply(lambda row: generate_candidate_sentences(row['kogemma'], row['kogemma_v2']), axis=1)
data['len'] = data.apply(lambda x: len(x['candidate']) if len(x['candidate']) > 1 else 0, axis = 1)
data['sent_len'] = data.apply(lambda x: len(x['kogemma'].split()), axis = 1)

In [ ]:
data['len'].value_counts()

## comparison

In [ ]:
def select_best_candidate(candidates: list, model, tokenizer, device: str, sentence1: str, sentence2: str) -> str:
    """
    후보 문장 리스트에서, 후보 수에 따라 다음과 같이 선택.
      - 후보가 1개이면: 바로 그 문장을 선택 (펄플렉시티 계산 없이)
      - 후보가 200개 이상이면: 두 원본 문장(sentence1, sentence2)만 펄플렉시티를 계산하여 더 낮은 값을 가진 문장을 선택
      - 그 외의 경우: 모든 후보 문장에 대해 펄플렉시티를 계산한 후, 가장 낮은 값을 가진 문장을 선택

    Returns:
        (best_candidate, best_ppl): 선택된 문장과 그 perplexity 값 (계산하지 않은 경우 None)
    """
    if len(candidates) == 1:
        # 후보가 단 1개이면, 바로 반환 (펄플렉시티 계산 없이)
        best_candidate = candidates[0]
        return best_candidate

    elif len(candidates) >= 300: # 할만해보여서 256까지 허용
        # 후보가 너무 많으면, 원본 두 문장끼리만 펄플렉시티를 계산.
        ppl1 = calculate_perplexity(sentence1, model, tokenizer, device)
        ppl2 = calculate_perplexity(sentence2, model, tokenizer, device)
        best_candidate = sentence1 if ppl1 < ppl2 else sentence2
        return best_candidate

    else:
        # 후보가 300개 미만인 경우: 모든 후보에 대해 펄플렉시티를 계산.
        best_candidate = None
        best_ppl = float("inf")
        for candidate in candidates:
            ppl = calculate_perplexity(candidate, model, tokenizer, device)
            if ppl < best_ppl:
                best_ppl = ppl
                best_candidate = candidate
        return best_candidate


In [ ]:
model.eval()
from tqdm import tqdm
tqdm.pandas()
data['best_sentence'] = data.progress_apply(lambda x: select_best_candidate(x['candidate'],
                                                                            model, tokenizer, device,
                                                                            x['kogemma'],
                                                                            x['kogemma_v2']), axis=1)

In [ ]:
import re

def clean_blank(text):
    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환
    return text

def restore_original_text(original_text, decoded_text):

    restored_text = ""
    decoded_idx = 0  # 해독된 텍스트에서 현재 참조 중인 문자 위치

    # 한글, 영어, 숫자, 공백, 문장부호만 허용하는 정규식
    valid_char_pattern = re.compile(r'[가-힣a-zA-Z0-9(),.!?\'\"\[\]\{\}:]')


    for char in original_text:
        if valid_char_pattern.match(char):
            # 유효한 문자일 경우, 해독된 문자로 대체
            if decoded_idx < len(decoded_text):
                restored_text += decoded_text[decoded_idx]
                decoded_idx += 1
            else:
                # 만약 해독된 텍스트가 끝났다면 원문 문자 그대로 추가
                restored_text += char
        else:
            # 특수문자 및 불완전한 한글은 원문 그대로 유지
            restored_text += char

    return restored_text

def check_double_blanks(text):
    return bool(re.search(r' {2,}', text))

def check_repeated_last_char(text):
    return bool(re.search(r'([가-힣])\1+$', text))

def remove_trailing_unks(text):

    cleaned_text = re.sub(r'(<unk>\s*)+$', '', text)
    return cleaned_text

def post_processing(data, data_chunked, result, test_flg = True):
  temp = data.copy()
  temp_chunked = data_chunked.copy()

  # for i in range(len(result)):
  #   result[i] = remove_trailing_unks(result[i])

  temp_chunked['result'] = result
  data_recovered = temp_chunked.groupby("index").agg(list).reset_index(drop=True)
  if test_flg:
    data_recovered.columns = ['kogemma','kogemma_v2', 'result']
    data_recovered['result'] = data_recovered['result'].apply(lambda x: ' '.join(x))
  else:
    data_recovered.columns = ['input_DEC1','output','input_DEC2']
    data_recovered['input_DEC2'] = data_recovered['input_DEC2'].apply(lambda x: ' '.join(x))

  temp['input'] = temp.apply(lambda x: clean_blank(x['input']), axis = 1)

  temp['result'] = data_recovered['result']
  temp['result'] = temp.apply(lambda x: clean_blank(x['result']), axis = 1)
  temp['result'] = temp.apply(lambda x: x['result'].replace(" ",""), axis= 1)

  temp['result'] = temp.apply(lambda x: restore_original_text(x['input'],x['result']), axis = 1)

  return temp

In [ ]:
test = pd.read_csv(path+'/test.csv')
sub = post_processing(test, data[['index','kogemma','kogemma_v2']], data['best_sentence'])
sub.drop('input', axis = 1, inplace = True)
sub.columns = ['ID', 'output']
sub.to_csv(path+'/kogemma vs kogemma_v2(submission).csv', index = False)

# result1 vs result2

## Results

In [ ]:
data1 = pd.read_csv(path+'/kogemma vs electra(submission).csv')
data2 = pd.read_csv(path+'/kogemma vs kogemma_v2(submission).csv')

## Cleaning Data

In [ ]:
def clean_text(text):
    # 1. 한국어(가-힣), 영어(a-zA-Z), 숫자(0-9), 공백(\s) 외 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s(),.!?\'\"\[\]\{\}:]', '', text)

    # 2. ㅋㅋ, ㅜㅜ, ㅠㅠ, ㅎㅎ 등과 같은 불완전한 한글 제거
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]', '', text)

    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환

    return text

def split_sentences(text):
    # 기본적인 문장 분리
    sentences = re.split(r'(?<=[.!?])\s', text)
    sentences = [sentence.strip() for sentence in sentences if sentence]  # 공백 제거 및 빈 요소 제거

    merged_sentences = []
    temp = ""
    open_bracket_count = 0  # 열린 괄호 개수

    for i, sentence in enumerate(sentences):
        # 괄호 개수 체크
        open_bracket_count += sentence.count("(")
        open_bracket_count -= sentence.count(")")

        # 문장이 15자 이하이거나, 괄호가 닫히지 않았다면 다음 문장과 결합
        if (len(sentence) <= 15 and i < len(sentences) - 1) or open_bracket_count > 0:
            temp += sentence + " "
        else:
            temp += sentence
            merged_sentences.append(temp)
            temp = ""  # temp 초기화

    if temp:  # 마지막 문장이 남아 있다면 추가
        merged_sentences.append(temp.strip())

    # 마지막 문장은 반드시 마지막에서 두 번째 문장과 결합
    if len(merged_sentences) > 1:
        merged_sentences[-2] += " " + merged_sentences[-1]
        merged_sentences.pop()  # 마지막 문장 제거 (이미 합쳐졌으므로)

    return merged_sentences

def clean_and_chunk_data(data):
    data['cleaned_output'] = data['output'].apply(clean_text)
    data['length'] = data['cleaned_output'].apply(len)
    max_length = data['length'].max()
    data['chunked_output'] = data['cleaned_output'].apply(split_sentences)
    data_chunked = data.explode('chunked_output')[['chunked_output']].reset_index(drop=False)
    data_chunked['len'] = data_chunked['chunked_output'].apply(len)
    data_chunked.drop(columns=['len'], inplace=True, errors='ignore')
    return data_chunked, max_length

# 각 데이터에 함수 적용
data1_chunked, max_length1 = clean_and_chunk_data(data1)
data2_chunked, max_length2 = clean_and_chunk_data(data2)

In [ ]:
data = pd.concat([data1_chunked, data2_chunked[['chunked_output']]], axis =1)
data.columns = ['index','result1','result2']
data.to_csv(path+"/results_chunked.csv", index=False)

## Candidates

In [ ]:
data = pd.read_csv(path+"/results_chunked.csv")

In [ ]:
from itertools import product

def generate_candidate_sentences(sentence1: str, sentence2: str) -> list:

    tokens1 = sentence1.split()
    tokens2 = sentence2.split()

    if len(tokens1) != len(tokens2):
        raise ValueError("두 문장의 어절 수가 동일해야 합니다.")

    candidate_tokens = []
    for t1, t2 in zip(tokens1, tokens2):
        # 어절이 동일하면 한 가지 선택지만, 다르면 두 가지 선택지를 사용합니다.
        candidate_tokens.append([t1] if t1 == t2 else [t1, t2])

    # Cartesian product를 사용해 모든 조합의 문장을 생성합니다.
    candidates = [" ".join(tokens) for tokens in product(*candidate_tokens)]
    return candidates

data['candidate'] = data.apply(lambda row: generate_candidate_sentences(row['result1'], row['result2']), axis=1)
data['len'] = data.apply(lambda x: len(x['candidate']) if len(x['candidate']) > 1 else 0, axis = 1)
data['sent_len'] = data.apply(lambda x: len(x['result1'].split()), axis = 1)

## comparison

In [ ]:
def select_best_candidate(candidates: list, model, tokenizer, device: str, sentence1: str, sentence2: str) -> str:
    """
    후보 문장 리스트에서, 후보 수에 따라 다음과 같이 선택.
      - 후보가 1개이면: 바로 그 문장을 선택 (펄플렉시티 계산 없이)
      - 후보가 200개 이상이면: 두 원본 문장(sentence1, sentence2)만 펄플렉시티를 계산하여 더 낮은 값을 가진 문장을 선택
      - 그 외의 경우: 모든 후보 문장에 대해 펄플렉시티를 계산한 후, 가장 낮은 값을 가진 문장을 선택

    Returns:
        (best_candidate, best_ppl): 선택된 문장과 그 perplexity 값 (계산하지 않은 경우 None)
    """
    if len(candidates) == 1:
        # 후보가 단 1개이면, 바로 반환 (펄플렉시티 계산 없이)
        best_candidate = candidates[0]
        return best_candidate

    elif len(candidates) >= 2000: # 할만해보여서 1024까지 허용
        # 후보가 너무 많으면, 원본 두 문장끼리만 펄플렉시티를 계산.
        ppl1 = calculate_perplexity(sentence1, model, tokenizer, device)
        ppl2 = calculate_perplexity(sentence2, model, tokenizer, device)
        best_candidate = sentence1 if ppl1 < ppl2 else sentence2
        return best_candidate

    else:
        # 후보가 2000개 미만인 경우: 모든 후보에 대해 펄플렉시티를 계산.
        best_candidate = None
        best_ppl = float("inf")
        for candidate in candidates:
            ppl = calculate_perplexity(candidate, model, tokenizer, device)
            if ppl < best_ppl:
                best_ppl = ppl
                best_candidate = candidate
        return best_candidate


In [ ]:
model.eval()
from tqdm import tqdm
tqdm.pandas()
data['best_sentence'] = data.progress_apply(lambda x: select_best_candidate(x['candidate'],
                                                                            model, tokenizer, device,
                                                                            x['result1'],
                                                                            x['result2']), axis=1)

In [ ]:
import re

def clean_blank(text):
    text = text.strip()  # 좌우 공백 제거
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백을 하나로 변환
    return text

def restore_original_text(original_text, decoded_text):

    restored_text = ""
    decoded_idx = 0  # 해독된 텍스트에서 현재 참조 중인 문자 위치

    # 한글, 영어, 숫자, 공백, 문장부호만 허용하는 정규식
    valid_char_pattern = re.compile(r'[가-힣a-zA-Z0-9(),.!?\'\"\[\]\{\}:]')


    for char in original_text:
        if valid_char_pattern.match(char):
            # 유효한 문자일 경우, 해독된 문자로 대체
            if decoded_idx < len(decoded_text):
                restored_text += decoded_text[decoded_idx]
                decoded_idx += 1
            else:
                # 만약 해독된 텍스트가 끝났다면 원문 문자 그대로 추가
                restored_text += char
        else:
            # 특수문자 및 불완전한 한글은 원문 그대로 유지
            restored_text += char

    return restored_text

def check_double_blanks(text):
    return bool(re.search(r' {2,}', text))

def check_repeated_last_char(text):
    return bool(re.search(r'([가-힣])\1+$', text))

def remove_trailing_unks(text):

    cleaned_text = re.sub(r'(<unk>\s*)+$', '', text)
    return cleaned_text

def post_processing(data, data_chunked, result, test_flg = True):
  temp = data.copy()
  temp_chunked = data_chunked.copy()

  # for i in range(len(result)):
  #   result[i] = remove_trailing_unks(result[i])

  temp_chunked['result'] = result
  data_recovered = temp_chunked.groupby("index").agg(list).reset_index(drop=True)
  if test_flg:
    data_recovered.columns = ['result1','result2', 'result']
    data_recovered['result'] = data_recovered['result'].apply(lambda x: ' '.join(x))
  else:
    data_recovered.columns = ['input_DEC1','output','input_DEC2']
    data_recovered['input_DEC2'] = data_recovered['input_DEC2'].apply(lambda x: ' '.join(x))

  temp['input'] = temp.apply(lambda x: clean_blank(x['input']), axis = 1)

  temp['result'] = data_recovered['result']
  temp['result'] = temp.apply(lambda x: clean_blank(x['result']), axis = 1)
  temp['result'] = temp.apply(lambda x: x['result'].replace(" ",""), axis= 1)

  temp['result'] = temp.apply(lambda x: restore_original_text(x['input'],x['result']), axis = 1)

  return temp

In [ ]:
test = pd.read_csv(path+'/test.csv')
sub = post_processing(test, data[['index','result1','result2']], data['best_sentence'])
sub.drop('input', axis = 1, inplace = True)
sub.columns = ['ID', 'output']
sub.to_csv(path+'/result1 vs result2(submission).csv', index = False)

# 최종 제출
아래 세 결과 중에 `kogemma_v1 vs kogemma_v2`로 결정
- kogemma vs electra (result1)
- kogemma_v1 vs kogemma_v2 (result2)
- result1 vs result2

In [ ]:
ensemble2 = pd.read_csv(path+'/kogemma vs kogemma_v2(submission).csv')

In [ ]:
ensemble2.to_csv(path+'/final_submission.csv', index = False)